In [1]:
import json
import pandas as pd

# Load the JSON file
with open("dev_tables.json", "r", encoding="utf-8") as f:
    data = json.load(f)

table_schemas = {}
dbs = []

for db in data:
    db_id = db.get("db_id", [])
    dbs.append(db_id)
    schema_promt = f"Below are the descriptions for the database {db_id}:\n"
    for table in db.get("table_names_original", []):
        # import csv file in dev_databases/{db_id}/database_description/{table}.csv
        try:
            table_desc = pd.read_csv(f"dev_databases/{db_id}/database_description/{table}.csv")
        except Exception as e:
            print(db_id, table, e)
        table_index = db["table_names_original"].index(table)
        schema_promt += f"[Table name]\n{table}\n[(Column name, description, type)]\n"
        for i, (tbl_idx, col_name) in enumerate(db.get("column_names_original", [])):
            if tbl_idx == table_index:
                # find the column description whose original_column_name is col_name
                try:
                    col_desc = table_desc[table_desc["original_column_name"] == col_name]["column_description"].values[0]
                except Exception as e:
                    col_desc = ""
                type_desc = db.get("column_types", [])[i]
                schema_promt += f"{col_name}, {col_desc}, {type_desc}\n"
        schema_promt += "\n"
    table_schemas[db_id] = schema_promt

In [3]:
print(table_schemas["debit_card_specializing"])

Below are the descriptions for the database debit_card_specializing:
[Table name]
customers
[(Column name, description, type)]
CustomerID, identification of the customer, integer
Segment, client segment, text
Currency, Currency, text

[Table name]
gasstations
[(Column name, description, type)]
GasStationID, Gas Station ID, integer
ChainID, Chain ID, integer
Country, nan, text
Segment, chain segment, text

[Table name]
products
[(Column name, description, type)]
ProductID, Product ID, integer
Description, Description, text

[Table name]
transactions_1k
[(Column name, description, type)]
TransactionID, Transaction ID, integer
Date, Date, date
Time, Time, text
CustomerID, Customer ID, integer
CardID, Card ID, integer
GasStationID, Gas Station ID, integer
ProductID, Product ID, integer
Amount, Amount, integer
Price, Price, real

[Table name]
yearmonth
[(Column name, description, type)]
CustomerID, Customer ID, integer
Date, Date, text
Consumption, consumption, real




In [7]:
from openai import OpenAI

results = {}
for db_id in dbs:
    results[db_id] = []

client = OpenAI()

#generate full prompt for queries
prompt = table_schemas["debit_card_specializing"]
prompt += f"Given the above database schema, convert the following pseudo-SQL query to a executable SQL:\n"
prompt += f"SELECT nationality WHERE customer spent 548.4 on 2012/8/24\nReturn a single SQL with no explanation.\nBegin!"
response = ""
try:
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
            "role": "user",
            "content": prompt
            }
        ],
        temperature=0
    )
    response = completion.choices[0].message.content
except Exception as e:
    pass
# print(query["question"])
# print(query["query"])
print(response)

```sql
SELECT c.Currency 
FROM customers c 
JOIN transactions_1k t ON c.CustomerID = t.CustomerID 
WHERE t.Amount = 548.4 AND t.Date = '2012-08-24';
```
